In [ ]:
# Upload images

from google.colab import files
from io import BytesIO
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

uploaded = files.upload()

images = []
for fn in uploaded.keys():
  # Convert the uploaded file to an OpenCV image
  image_stream = BytesIO(uploaded[fn])
  image = Image.open(image_stream).convert('RGB')
  open_cv_image = np.array(image)
  # Convert RGB to BGR
  open_cv_image = open_cv_image[:, :, ::-1].copy()
  images.append(open_cv_image)


Saving 1.jpg to 1.jpg
Saving 2.jpg to 2.jpg
Saving 3.PNG to 3.PNG
Saving 4.PNG to 4.PNG
Saving 5.PNG to 5.PNG
Saving 6.PNG to 6.PNG
Saving 7.PNG to 7.PNG
Saving 8.jpg to 8.jpg
Saving 9.PNG to 9.PNG
Saving 10.PNG to 10.PNG
Saving 11.PNG to 11.PNG
Saving 12.PNG to 12.PNG
Saving 13.PNG to 13.PNG
Saving 14.jpg to 14.jpg
Saving 15.PNG to 15.PNG
Saving 16.PNG to 16.PNG
Saving 17.PNG to 17.PNG
Saving 18.jpg to 18.jpg
Saving 19.jpg to 19.jpg
Saving 20.jpg to 20.jpg


In [ ]:
# Process images and identify sky pixels

def process_image(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue, saturation, value = cv2.split(hsv_image)

    # Apply adaptive thresholding to the Value channel
    value_thresh = cv2.adaptiveThreshold(value, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Identify areas with low saturation, such as clouds
    low_saturation_areas = cv2.inRange(saturation, 0, 50)

    def create_combined_color_mask(hsv_image, color_bounds):
        # Initialize an empty mask
        combined_color_mask = np.zeros(hsv_image.shape[:2], dtype="uint8")

        # Iterate through each color range
        for (lower_bound, upper_bound) in color_bounds:
            # Generate mask for current color range and combine it with the existing mask
            color_mask = cv2.inRange(hsv_image, np.array(lower_bound), np.array(upper_bound))
            combined_color_mask = cv2.bitwise_or(combined_color_mask, color_mask)

        return combined_color_mask

    # Define color ranges to filter out
    color_bounds = [
        ([20, 100, 100], [85, 255, 255]),  # Yellow to Green
        ([10, 50, 20], [20, 255, 255]),    # Brown to Orange
        ([0, 50, 50], [10, 255, 255]),     # Lower Red hues (including Light Red)
        ([125, 50, 50], [180, 255, 255]),  # Upper Red hues (including Light Red)
        ([90, 25, 75], [110, 80, 255]),   # Light Blue
    ]

    # Create the combined color mask
    color_mask = create_combined_color_mask(hsv_image, color_bounds)


    # Filter out these colors from the sky mask
    sky_mask = cv2.bitwise_and(value_thresh, value_thresh, mask=cv2.bitwise_not(color_mask))

    # Combine the low saturation mask with the sky mask
    combined_mask = cv2.bitwise_and(sky_mask, sky_mask, mask=cv2.bitwise_not(low_saturation_areas))

    # Remove very bright areas (like the sun)
    bright_thresh = 245  # Adjust as needed
    very_bright_areas = cv2.inRange(value, bright_thresh, 255)
    combined_mask = cv2.bitwise_and(combined_mask, combined_mask, mask=cv2.bitwise_not(very_bright_areas))

    # Apply morphological operations
    kernel = np.ones((5,5), np.uint8)
    combined_mask = cv2.erode(combined_mask, kernel, iterations=1)
    combined_mask = cv2.dilate(combined_mask, kernel, iterations=3)

    # Apply edge detection on the combined mask
    edges = cv2.Canny(combined_mask, 100, 200)

    # Combine the mask with the original image
    result = cv2.bitwise_and(image, image, mask=combined_mask)

    return result, edges

# Process each image
processed_images_and_edges = [process_image(img) for img in images]

# Display results along with edges
for original, (processed, edges) in zip(images, processed_images_and_edges):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')

    plt.subplot(1, 3, 2)
    plt.imshow(cv2.cvtColor(processed, cv2.COLOR_BGR2RGB))
    plt.title('Sky Pixels Identified')

    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install gradio==3.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 sha256=d5384c6bd23f1eed6fe1e3d2916988d89920b9803d4cf11931599078e354d9d6

In [1]:
# Create Gradio demo

import gradio as gr

def gradio_process_image(image):
    # Convert the image from RGB (used by Gradio) to BGR (used by OpenCV)
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Process the image using our function
    processed_image, _dh = process_image(image_bgr)

    # Convert the processed image back to RGB for display in Gradio
    processed_image_rgb = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)

    return processed_image_rgb

# Create the Gradio interface
iface = gr.Interface(
    fn = gradio_process_image,
    inputs = gr.inputs.Image(type="numpy", label="Upload an Image"),
    outputs = gr.outputs.Image(type="numpy", label="Sky Identified"),
    title="Sky Pixel Identification",
    description="Upload an image to identify sky pixels."
)

# Launch the app
iface.launch(share=True)


ModuleNotFoundError: No module named 'gradio'